# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage
from langchain import Tool
from langchain.memory import ConversationBufferMemory

llm_high = ChatAnthropic(model="claude-3-5-sonnet-20240620")


In [2]:


messages = [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
]


In [3]:


parser = StrOutputParser()

chain =llm_high|parser
chain.invoke(messages)

'Your name is Bob, based on your introduction.'

In [ ]:


# ConversationBufferMemory 초기화
memory = ConversationBufferMemory()

# LLM Tool 정의: 버퍼 메모리를 사용하여 대화 히스토리를 자동 관리
def llm_response_tool_with_buffer(user_input):
    # 이전 대화 히스토리를 포함한 메시지 생성
    previous_messages = memory.chat_memory.messages
    
    # 새로운 사용자 메시지 추가
    memory.chat_memory.add_message(HumanMessage(content=user_input))
    
    # LLM 호출하여 응답 생성 (이전 히스토리 포함)
    response = llm.invoke(previous_messages + [HumanMessage(content=user_input)])
    
    # LLM 응답을 메모리에 추가
    memory.chat_memory.add_message(AIMessage(content=response.content))
    
    # 최종 응답 반환
    return response.content

# Tool 객체 생성
llm_tool_with_buffer = Tool(
    name="LLM Tool with Buffer Memory",
    func=llm_response_tool_with_buffer,
    description="대화 히스토리를 자동 관리하여 LLM에 응답을 요청하는 도구입니다."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType

# OpenAI 모델 및 Tool이 포함된 Agent 생성
agent = initialize_agent(
    tools=[llm_tool_with_buffer],
    llm=llm_high,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory  # 대화 메모리를 포함하여 이전 대화 맥락 유지
)


In [ ]:
from langchain import Tool, OpenAI
from langchain.schema import HumanMessage

# OpenAI 모델 초기화
llm = OpenAI(model="gpt-3.5-turbo")

# 1. 분석 Tool 정의: LLM을 사용해 요청 유형을 결정
def analyze_with_llm_tool(user_input):
    analysis_prompt = (
        "사용자의 요청을 분석하여 필요한 응답의 유형을 결정하세요. "
        "과거의 사건에 대해 언급하였다면 'recall' "
        "그렇지 않은 일반 대화라면 'normal', 인식 불가라면 'undefined'를 반환하세요.\n\n"
        f"요청: {user_input}"
    )
    
    # LLM으로 분석 요청 
    analysis_response = llm.invoke([HumanMessage(content=analysis_prompt)])
    
    # 분석 결과에서 결정된 유형 추출
    if "계산" in analysis_response.content:
        return "calculator"
    elif "상담" in analysis_response.content:
        return "advice"
    else:
        return "unknown"

# Tool 객체 생성
analysis_tool = Tool(
    name="LLM Analysis Tool",
    func=analyze_with_llm_tool,
    description="요청의 유형을 분석하여 어떤 에이전트를 사용할지 결정합니다."
)

# 2. 관리용 Agent 함수: 분석 Tool을 사용하여 Agent 선택
def manage_agents(user_input):
    # 분석 Tool을 사용해 요청 유형 결정
    agent_type = analysis_tool.func(user_input)
    
    # 분석 결과에 따라 적절한 Agent 호출
    if agent_type == "calculator":
        response = calculator_agent({"input": user_input})["output"]
    elif agent_type == "advice":
        response = advice_agent({"input": user_input})["output"]
    else:
        response = "죄송합니다, 요청을 처리할 수 없습니다."
    
    return response

# 관리용 Agent 실행 예제
def chatbot():
    print("관리용 에이전트 챗봇입니다. '종료'라고 입력하면 대화가 끝납니다.")
    while True:
        user_input = input("User: ")
        if user_input.lower() == "종료":
            print("대화를 종료합니다.")
            break
        output = manage_agents(user_input)
        print("Bot:", output)

# 챗봇 실행
chatbot()
